# Part 1 using list insertion/deletion

In [ ]:
my_input = "389125467"
my_input = "459672813"

cups = list(map(int, my_input))
MAX_VALUE = max(cups)

In [ ]:
current_index = 0
moves = 0

for _ in range(100):
    moves += 1
    current_value = cups[current_index]

#     print(f"-- move {moves} --")
#     print("cups:", " ".join(str(cup) if cup != current_value else f"({cup})" for cup in cups))

    # Remove 3 next cups
    pick_up = cups[current_index + 1: current_index + 4]
    del cups[current_index + 1: current_index + 4]

    if len(pick_up) < 3:
        left = 3 - len(pick_up)
        pick_up += cups[:left]
        del cups[:left]
        current_index = current_index - left

#     print("pick up:", ", ".join(str(cup) for cup in pick_up))

    # Find target value and -index
    target_value = (current_value - 2) % MAX_VALUE + 1
    while target_value in pick_up:
        target_value = (target_value - 2) % MAX_VALUE + 1
    target_index = cups.index(target_value)

#     print("destination:", target_value, "\n")

    # Insert cups after target index
    cups[target_index + 1: target_index + 1] = pick_up

    # If the current value shifted right, update index
    if target_index < current_index:
        current_index += 3

    # Next current is to the right
    current_index = (current_index + 1)  % len(cups)

In [ ]:
cups

In [ ]:
index = cups.index(1)
solution = []
for _ in range(len(cups) - 1):
    index = (index + 1) % len(cups)
    solution.append(cups[index])

"".join(map(str, solution))

# Part 2 - Using dictionary / linked list

In [ ]:
from dataclasses import dataclass

@dataclass
class Cup:
    value: int
    next = None

    def __repr__(self):
        if self.next is None:
            next_value = "None"
        else:
            next_value = self.next.value
        return f"{self.value} -> {next_value}"

In [ ]:
def create_linked_lookup(cup_values):

    # Initialize variables
    lookup = dict()
    previous_value = None
    max_value = 0

    # Create a linked list and lookup dictionary of the cups 
    for value in cup_values:
        # Store the highest observed value
        max_value = max(max_value, value)

        # Create a cup and link the previous cup to this one
        lookup[value] = Cup(value)
        if previous_value is None:
            first_value = value
        else:
            lookup[previous_value].next = lookup[value]
        previous_value = value

    # Finally, connect the last cup with the first
    lookup[value].next = lookup[first_value]
    
    return lookup, first_value, max_value

def remove_next_3_after(lookup, value):
    pick_up = lookup[value].next
    lookup[value].next = pick_up.next.next.next
    pick_up.next.next.next = None
    return pick_up

def insert_pick_up_after(lookup, pick_up, value):
    next_cup = lookup[value].next
    lookup[value].next = pick_up
    pick_up.next.next.next = next_cup

def perform_moves(num, lookup, current_value, MAX_VALUE):
    for i in range(num):

        # Remove 3 cups
        pick_up = remove_next_3_after(lookup, current_value)

        # Find insert-destination
        pick_up_values = {pick_up.value, pick_up.next.value, pick_up.next.next.value}
        destination = (current_value - 2) % MAX_VALUE + 1
        while destination in pick_up_values:
            destination = (destination - 2) % MAX_VALUE + 1

        # Insert removed cups
        insert_pick_up_after(lookup, pick_up, destination)

        # Set new current value
        current_value = lookup[current_value].next.value
    
    return lookup

## Use this solution to solve part 1

In [ ]:
def part1(puzzle_input: str):
    lookup = perform_moves(100, *create_linked_lookup(map(int, puzzle_input)))
    value = 1
    numbers = []
    while lookup[value].next.value != 1:
        value = lookup[value].next.value
        numbers.append(value)
    
    return "".join(map(str, numbers))

In [ ]:
assert "67384529" == part1("389125467")
assert "68245739" == part1("459672813")  # My puzzle input

## Find solution for part 2

In [ ]:
from itertools import chain

def part2(puzzle_input: str):
    lookup = perform_moves(
        10_000_000,
        *create_linked_lookup(chain(map(int, puzzle_input), range(10, 1_000_001)))
    )
    return lookup[1].next.value * lookup[1].next.next.value

In [ ]:
assert 149245887792 == part2("389125467")
assert 219634632000 == part2("459672813")  # My puzzle input